## Lets start by opening our directory and concatating all files and their content

In [9]:
import os
import pandas as pd
from numpy import NaN




from src.data.dataMakers import dataMakers
dm = dataMakers()
fighterFrame = dm.make()

ModuleNotFoundError: No module named 'src'

In [1]:
#get list of all the text files in the directory ./names
files = [f for f in os.listdir("./names") if os.path.isfile(os.path.join("./names", f))]

#merge all files and their content into a pandas dataFrame
namesFrame = pd.DataFrame()
pieces = []
for file in files:
    pieces.append(pd.read_csv(os.path.join("./names",file), names=['name','sex','births']))
    
data = pd.concat(pieces,ignore_index=True)
namesFrame = data[['name','sex','births']]
malesFrame = namesFrame[(namesFrame.sex == 'M')].groupby('name')
femalesFrame = namesFrame[(namesFrame.sex == 'F')].groupby('name')

print(malesFrame.get_group('Gloria').births.sum())

1738


In order to improve the classification, we are going to remove unisex names from the sets

In [2]:
#Get the complement between males and females, the complement of the sets are the unisex_names
females_set = set(namesFrame[namesFrame.sex == 'F']['name'])
males_set = set(namesFrame[namesFrame.sex == 'M']['name'])
unisex_names = set(set(females_set) & set(males_set))

#Remove unisex names from each set
unique_males = males_set.difference(unisex_names)
unique_females = females_set.difference(unisex_names)

In [3]:
#assign a probability to each unisex name
ambiguous_names = set()
count = 0

for element in unisex_names:
    
    male_births = malesFrame.get_group(element).births.sum()
    female_births = femalesFrame.get_group(element).births.sum()
    total_births = male_births + female_births
    probability_m = male_births/total_births
    probability_f = 1- probability_m
    
    #only add the name back to the male or female set if they meet the probability threshold
    threshold = .97
    if probability_m >= threshold:
        unique_males.add(element)
        
    elif probability_f >= threshold:
        unique_females.add(element)
    else:
        ambiguous_names.add(element)


In [4]:
def sex_classifyer(name):
    if name in unique_males:
        return 'M'
    elif name in unique_females:
        return 'F'
    else:
        return NaN

In [ ]:
#validate the accuracy of the function with female set of names
